# Data Prep

<font color="purple">fctProduction DW + v1 d365 Account Data</font>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./Data/wdw_Production_2020_to_2022_FY.csv',low_memory=False)

# handle unicode issues with this d365 sourced csv
# its a microsoft file so it makes sense it has the different encoding
# iterate over our d365 account files (dictated by the advanced find import limits of 100k rows)
d365_paths = ['./Data/d365_adv_find_Active_A-L.csv', './Data/d365_adv_find_Active_M-Z.csv','./Data/d365_adv_find_Active_nonAlphaStart.csv']
dfs_d365 = []
for p in d365_paths:
    try:
        tmp = pd.read_csv(p, low_memory=False, encoding='utf-8')
    except UnicodeDecodeError:
        tmp = pd.read_csv(p, low_memory=False, encoding='windows-1252')
    #display(tmp.head(2))
    dfs_d365.append(tmp)
    del tmp
df2 = pd.concat(dfs_d365)


In [3]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130374 entries, 0 to 130373
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   JobId                130374 non-null  int64  
 1   AssociateID          130374 non-null  int64  
 2   OfficeCity           130353 non-null  object 
 3   OfficeState          130233 non-null  object 
 4   OfficeZip            130233 non-null  float64
 5   OfficeName           130353 non-null  object 
 6   AccountGUID          130374 non-null  object 
 7   AccountName          130374 non-null  object 
 8   AccountCity          130341 non-null  object 
 9   AccountState         130341 non-null  object 
 10  AccountZip           130317 non-null  object 
 11  AccountNaicsName     130356 non-null  object 
 12  MarketName           130374 non-null  object 
 13  ServiceName          130374 non-null  object 
 14  ServiceLineName      130374 non-null  object 
 15  IndustryName     

In [4]:
#display(df.info())
display(df2.info())
#df2['Assets'].value_counts()
#df2[df2['# of Locations'].isna() == True]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154514 entries, 0 to 1659
Data columns (total 9 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   (Do Not Modify) Account ID    154514 non-null  object 
 1   (Do Not Modify) Row Checksum  154514 non-null  object 
 2   (Do Not Modify) Modified On   154514 non-null  object 
 3   Account Name                  154514 non-null  object 
 4   # of Locations                5693 non-null    float64
 5   Annual Sales                  33752 non-null   object 
 6   Assets                        1915 non-null    object 
 7   Employees                     24232 non-null   object 
 8   SEC/Publicly Traded           154513 non-null  object 
dtypes: float64(1), object(8)
memory usage: 11.8+ MB


None

In [5]:
# column refinements df 
df['JobTaxComplexity'].replace({np.nan:'Not Applicable'}, inplace=True) # fill complexity to not applicable
df['Cost'].replace({np.nan:0}, inplace=True) # replace cost with 0 when its blank


# SHOULD DO THIS IS THE SQL DATA ACQUISITION QUERY
# only consider clients because those prospect jobs have no wip or any real financial info
# df = df[df['AccountRelationshipType'] == 'Client']
# df.drop(columns=['AccountRelationshipType'], inplace=True)

# number of unique cols where we don't have a -9 (unknown) associate
display(df[df['AssociateID'] != -9].JobId.nunique())
# number of uniqu cols with known associates
display(df[df['AssociateID'] == -9].shape[0])
display(df.shape[0])
display(df['JobId'].nunique())

25532

33252

130374

29134

In [6]:
# identifying unique work items
pd.concat([df['JobId'], df['ServiceName'], df['JobName'], df['CalendarYear']]).nunique()

41458

In [7]:
# checking our categories, this is just an example
df2['Annual Sales'].value_counts(), df2['Annual Sales'].isna().sum()

(5M - 9.9M        8902
 50M - 99M        3916
 10M - 24.9M      3332
 Unknown          3236
 1M - 2.4M        2681
 $200K - $499K    2607
 Under $200K      2506
 $500K - $999K    2431
 2.5M - 4.9M      2239
 25M - 49M        1022
 100M - 249M       474
 250M - 499M       197
 750M - 999M        68
 500M - 749M        61
 1B - 1.49B         37
 2B +               28
 1.5B - 1.9B        15
 Name: Annual Sales, dtype: int64,
 120762)

In [8]:
for col in df2.columns:
    null_pct = (df2[col].isna().sum()/df2.shape[0])*100
    if null_pct > 0.:
        print(f"{col}: {null_pct:,.2f}% nulls")

# of Locations: 96.32% nulls
Annual Sales: 78.16% nulls
Assets: 98.76% nulls
Employees: 84.32% nulls
SEC/Publicly Traded: 0.00% nulls


In [9]:
# column refinements df2
# should this be done in the pipeline????
df2['# of Locations'].replace({np.nan:0.},inplace=True) # encode unknown as a huge number so its categorical
df2['Annual Sales'].fillna('Unknown',inplace=True)# mark as unknown
df2['Assets'].fillna('Unknown', inplace=True)# mark as unknown
df2['Employees'].replace({np.nan:'Unknown', '9-Jan':'1 - 9'
                          , '24-Oct':'10 - 24'},inplace=True) #adjust some of the employee numbers that are interpolated as dates 
df2['SEC/Publicly Traded'].fillna('No',inplace=True) # fill the rare case of missing with No


df2.drop(columns=['(Do Not Modify) Row Checksum','(Do Not Modify) Modified On'], inplace=True)

---
## Now that the preliminary data cleanup has been completed, we can group our fctProduction oriented data to the Job Level (by JobKey) and then join it to our d365 account data.

In [10]:
# group by for the wdw df(need to revise sql query
# SHOULD THE GROUP BY EXCLUDE NULL ASSOCIATE KEYS ? E.G.
# rollup associate sublevel to job (removing AssociateKey as a grouper)
df_grouped = df.groupby(by=['JobId','CalendarYear','OfficeCity', 'OfficeState', 'OfficeZip',
       'OfficeName', 'AccountGUID','AccountName', 'AccountCity', 'AccountState',
       'AccountZip', 'AccountNaicsName',
       'MarketName', 'ServiceName', 'ServiceLineName', 'IndustryName',
       'JobTaxComplexity', 'JobName']).agg(
    {
        'ChargeHours':'sum', 'ProductionRevActual':'sum',
        'Cost':'sum', 'NumAssociatesOnJob':'sum'
    }
)
# reset our index
df_grouped = df_grouped.reset_index()

# drop rows where we have null values in the target
display(df_grouped.shape)
# DEPRECATED: only need the below if you do something in agg() like {'ChargeHours':['sum', 'mean']} which creates multiple levels on the column axis
#df_grouped.columns = ['_'.join(col) if len(col[-1]) > 0 else col[0]  for col in df_grouped.columns.values]
df_grouped.head()

(36937, 22)

,JobId,CalendarYear,OfficeCity,OfficeState,OfficeZip,OfficeName,AccountGUID,AccountName,AccountCity,AccountState,...,MarketName,ServiceName,ServiceLineName,IndustryName,JobTaxComplexity,JobName,ChargeHours,ProductionRevActual,Cost,NumAssociatesOnJob
0,5418,2020,Wausau,WI,54403.0,"Wausau, WI (30)",26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,Rothschild,WI,...,North Central WI,Accounting assistance - A&A,Audit & Accounting (10),Mfg. Retail Distribution,Not Applicable,Accounting assistance - A&A-2013,0.00,0.00,0.00,1
1,5430,2020,Wausau,WI,54403.0,"Wausau, WI (30)",2cbe5bdb-5c6c-e411-ab11-a45d36fcbf98,Bay Area Nephrology SC,Green Bay,WI,...,North Central WI,Accounting assistance - A&A,Audit & Accounting (10),Healthcare,Not Applicable,Accounting assistance - A&A-2013,1.40,140.00,35.13,2
2,5433,2020,Wausau,WI,54403.0,"Wausau, WI (30)",1ac15bdb-5c6c-e411-ab11-a45d36fcbf98,Bay Promotions LLC,Green Bay,WI,...,North Central WI,Accounting assistance - A&A,Audit & Accounting (10),Mfg. Retail Distribution,Not Applicable,Accounting assistance - A&A-2013,2.00,544.00,192.38,2
3,5470,2020,Wausau,WI,54403.0,"Wausau, WI (30)",52be5bdb-5c6c-e411-ab11-a45d36fcbf98,Car Partz LLC,Rhinelander,WI,...,North Central WI,Accounting assistance - A&A,Audit & Accounting (10),Mfg. Retail Distribution,Not Applicable,Accounting assistance - A&A-2013,4.25,563.75,139.43,2
4,5471,2020,Wausau,WI,54403.0,"Wausau, WI (30)",52be5bdb-5c6c-e411-ab11-a45d36fcbf98,Car Partz LLC,Rhinelander,WI,...,North Central WI,1120S,Tax (20),Mfg. Retail Distribution,Basic,1120S-2013,4.00,871.00,202.29,6


In [11]:
df_grouped.nunique()

JobId                  29070
CalendarYear               3
OfficeCity                41
OfficeState               15
OfficeZip                 41
OfficeName                41
AccountGUID            13463
AccountName            13463
AccountCity             2324
AccountState              52
AccountZip              3153
AccountNaicsName         834
MarketName                16
ServiceName              134
ServiceLineName            2
IndustryName               9
JobTaxComplexity           4
JobName                12229
ChargeHours             5421
ProductionRevActual    16076
Cost                   22125
NumAssociatesOnJob        32
dtype: int64

In [12]:
df2.head()

,(Do Not Modify) Account ID,Account Name,# of Locations,Annual Sales,Assets,Employees,SEC/Publicly Traded
0,c9bd0a08-5a6c-e411-ab11-a45d36fcbf98,A & A Furniture Company Inc,0.0,Unknown,Unknown,Unknown,No
1,aaa72327-847f-e611-80f0-5065f38b11e1,A & A Metals Inc,0.0,Unknown,Unknown,Unknown,No
2,6800531d-f8b6-e511-80e8-3863bb35ece0,A & B Machine Works Inc,0.0,$500K - $999K,Unknown,1 - 9,No
3,c29ab230-6400-e811-810b-5065f38b81e1,A & B Management Corp,0.0,5M - 9.9M,Unknown,Unknown,No
4,43719c9f-5d6c-e411-8419-6c3be5a82838,A & B Marketing Inc,0.0,10M - 24.9M,Unknown,10 - 24,No


In [13]:
merged = pd.merge(left=df_grouped, right=df2, how='inner',left_on='AccountGUID', right_on='(Do Not Modify) Account ID')
merged.shape

(35784, 29)

In [14]:
# set our merged df back to df
df = merged
display(dict(df.nunique()))
display(df.columns)

{'JobId': 28152,
 'CalendarYear': 3,
 'OfficeCity': 41,
 'OfficeState': 15,
 'OfficeZip': 41,
 'OfficeName': 41,
 'AccountGUID': 13020,
 'AccountName': 13020,
 'AccountCity': 2280,
 'AccountState': 52,
 'AccountZip': 3098,
 'AccountNaicsName': 831,
 'MarketName': 16,
 'ServiceName': 134,
 'ServiceLineName': 2,
 'IndustryName': 9,
 'JobTaxComplexity': 4,
 'JobName': 11876,
 'ChargeHours': 5334,
 'ProductionRevActual': 15713,
 'Cost': 21552,
 'NumAssociatesOnJob': 32,
 '(Do Not Modify) Account ID': 13020,
 'Account Name': 13020,
 '# of Locations': 27,
 'Annual Sales': 17,
 'Assets': 17,
 'Employees': 12,
 'SEC/Publicly Traded': 2}

Index(['JobId', 'CalendarYear', 'OfficeCity', 'OfficeState', 'OfficeZip',
       'OfficeName', 'AccountGUID', 'AccountName', 'AccountCity',
       'AccountState', 'AccountZip', 'AccountNaicsName', 'MarketName',
       'ServiceName', 'ServiceLineName', 'IndustryName', 'JobTaxComplexity',
       'JobName', 'ChargeHours', 'ProductionRevActual', 'Cost',
       'NumAssociatesOnJob', '(Do Not Modify) Account ID', 'Account Name',
       '# of Locations', 'Annual Sales', 'Assets', 'Employees',
       'SEC/Publicly Traded'],
      dtype='object')

---

# Drop identifier cols so the analysis is properly ambigious

In [15]:
# 
print('count of unique job keys as a proportion of rows\n(aka what percentage of records belong to one JobId,JobName,ServiceName,CalendarYear')

print(f'\t\t{(pd.concat([df.JobId, df.JobName, df.ServiceName, df.CalendarYear]).nunique() / df.shape[0])*100:,.2f}%')
print(f'shape: {df.shape}')

count of unique job keys as a proportion of rows
(aka what percentage of records belong to one JobId,JobName,ServiceName,CalendarYear
		112.03%
shape: (35784, 29)


In [16]:
d = dict(df.JobId.value_counts())
d2 = {k:v for k,v in d.items() if v > 1}
# d2 are the multirow records
evaluate_dups = df[df['JobId'].isin([int(x) for x in d2.keys()])]
#evaluate_dups[['(Do Not Modify) Account ID', 'JobId','AccountName', 'Account Name','Annual Sales', 'Assets', 'Employees',
#       'SEC/Publicly Traded', 'Relationship Maturity']]#.to_csv('./Data/issues.csv',index=False)
evaluate_dups.head()#.to_csv('./test.csv')

,JobId,CalendarYear,OfficeCity,OfficeState,OfficeZip,OfficeName,AccountGUID,AccountName,AccountCity,AccountState,...,ProductionRevActual,Cost,NumAssociatesOnJob,(Do Not Modify) Account ID,Account Name,# of Locations,Annual Sales,Assets,Employees,SEC/Publicly Traded
1,291345,2020,Wausau,WI,54403.0,"Wausau, WI (30)",26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,Rothschild,WI,...,175.50,29.63,4,26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,0.0,5M - 9.9M,Unknown,Unknown,No
2,291345,2021,Wausau,WI,54403.0,"Wausau, WI (30)",26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,Rothschild,WI,...,72.00,16.74,2,26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,0.0,5M - 9.9M,Unknown,Unknown,No
4,661912,2020,Wausau,WI,54403.0,"Wausau, WI (30)",26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,Rothschild,WI,...,739.00,144.95,2,26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,0.0,5M - 9.9M,Unknown,Unknown,No
5,661912,2021,Wausau,WI,54403.0,"Wausau, WI (30)",26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,Rothschild,WI,...,317.50,49.59,3,26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,0.0,5M - 9.9M,Unknown,Unknown,No
6,662755,2020,Wausau,WI,54403.0,"Wausau, WI (30)",26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,Rothschild,WI,...,4323.81,701.97,4,26be5bdb-5c6c-e411-ab11-a45d36fcbf98,B & T of Wausau Inc,0.0,5M - 9.9M,Unknown,Unknown,No


In [17]:
drop_cols = ['JobId','(Do Not Modify) Account ID', 'AccountGUID', 'Account Name', 'AccountName']

df= df.drop(columns=drop_cols)


In [18]:
# save output for use in EDA/modeling
df.to_csv('./Data/clean_data.csv', index=False)